# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
import pickle

In [2]:
!pip install transformers==4.19.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Connect to GPU

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda")

In [4]:
!nvidia-smi

Wed Jun 15 11:44:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install model

In [5]:
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained('AnyaSchen/rugpt3_mayakovskij').to(DEVICE)

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/983 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

# Fine-tuning step-by-step

## Add special tokens

In [7]:
SPECIAL_TOKENS = {'bos_token' : "<bos>", "eos_token" :"<eos>", 'pad_token':'<pad>'}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 1024)

## Get data

In [8]:
!wget https://www.dropbox.com/s/b6q4tokw0skjsfz/dataset.pkl?dl=0 -O dataset.pkl

--2022-06-14 09:21:54--  https://www.dropbox.com/s/b6q4tokw0skjsfz/dataset.pkl?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b6q4tokw0skjsfz/dataset.pkl [following]
--2022-06-14 09:21:54--  https://www.dropbox.com/s/raw/b6q4tokw0skjsfz/dataset.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5e16d059c9ba17dd0b89f5e41f.dl.dropboxusercontent.com/cd/0/inline/BnJAdsI_m4jgrlc_NEUp-ksOm9Ch0zRbX8n0t3V0wf0oXO7adxt389h7Br2t0fp3KIg4sX-UEvmrKbCYzHX5YW3TVqRWUpfYmwQKRSkLS4mp64mPHS-MkHEojwpfebBTgcpaQBfEAcOqB8A4LQWM_zv6Vb_Z-7eHJMJB4MykIu0PuQ/file# [following]
--2022-06-14 09:21:54--  https://uc5e16d059c9ba17dd0b89f5e41f.dl.dropboxusercontent.com/cd/0/inline/BnJAdsI_m4jgrlc_NEUp-ksOm9Ch0zRbX8n0t3V0wf0oXO7adxt389h7Br2t0fp3KI

In [9]:
with open('dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

## Greate a Dataset

In [10]:
import torch
torch.manual_seed(42)
from torch.utils.data import Dataset # this is the pytorch class import

class myDataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=400):

    self.tokenizer = tokenizer # the gpt2 tokenizer we instantiated
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:
      """
      This loop will iterate through each entry in the flavour text corpus.
      For each bit of text it will prepend it with the start of text token,
      then append the end of text token and pad to the maximum length with the 
      pad token. 
      """

      encodings_dict = tokenizer('<bos>'+ txt + '<eos>', 
                                 truncation=True, 
                                 max_length=max_length, 
                                 padding="max_length")
      
      """
      Each iteration then appends either the encoded tensor to a list,
      or the attention mask for that encoding to a list. The attention mask is
      a binary list of 1's or 0's which determine whether the langauge model
      should take that token into consideration or not. 
      """
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return {
        'input_ids': self.input_ids[idx],
        'attn_masks': self.attn_masks[idx]
    }

In [11]:
train_dataset = myDataset(dataset['mayakovskij'], tokenizer)

## Add Datacollator

In [12]:
from transformers import TextDataset, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
train_dataset[9]

{'attn_masks': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1,

## Training

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/mayak_checkouts/", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=40, # number of training epochs
    per_device_train_batch_size=2, # batch size for training
    per_device_eval_batch_size=2,  # batch size for evaluation
    warmup_steps=45,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=1, # to make "virtual" batch size larger
    save_steps = 1000
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [15]:
trainer.train(resume_from_checkpoint = '/content/drive/MyDrive/mayak_checkouts/checkpoint-7000')

Loading model from /content/drive/MyDrive/mayak_checkouts/checkpoint-7000).
***** Running training *****
  Num examples = 404
  Num Epochs = 40
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 8080
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 34
  Continuing training from global step 7000
  Will skip the first 34 epochs then the first 132 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/132 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: attn_masks. If attn_masks are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Step,Training Loss
7500,0.737900
8000,0.712700


Saving model checkpoint to ./drive/MyDrive/mayak_checkouts/checkpoint-8000
Configuration saved in ./drive/MyDrive/mayak_checkouts/checkpoint-8000/config.json
Model weights saved in ./drive/MyDrive/mayak_checkouts/checkpoint-8000/pytorch_model.bin


Step,Training Loss
7500,0.737900
8000,0.712700




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=8080, training_loss=0.09648282504317784, metrics={'train_runtime': 912.0857, 'train_samples_per_second': 17.718, 'train_steps_per_second': 8.859, 'total_flos': 1.1724846268416e+16, 'train_loss': 0.09648282504317784, 'epoch': 40.0})

## Save models

In [16]:
trainer.save_model('/content/drive/MyDrive/mayak_model/')

Saving model checkpoint to /content/drive/MyDrive/mayak_model/
Configuration saved in /content/drive/MyDrive/mayak_model/config.json
Model weights saved in /content/drive/MyDrive/mayak_model/pytorch_model.bin


# Generation

In [7]:
# Пример вероятностного сэмплирвоания с ограничением
text = "Послушайте"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=3,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=400,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Послушайте:
в Петрограде
          и в Москве
                буржуи и мещане,
богачи и нищие,
            все
         человечество —
обменивается
          именем
         Пуанкаре*.
Он
        лежит
    в парижской больнице,
лежит
          и стонет:
               «Железо!
          Железо!
              Бельмо!»
А рядом —
    толпа
         и мещане.
Каждый с утра
         и весь день
обменивается
         именем
         Пуанкаре.
Каждый с утра
         обменивается
          
